In [2]:
import sys
import numpy as np
import pandas as pd
import scipy as sp
import sklearn

In [3]:
import nltk
import gensim.corpora
import pickle

In [4]:
from nltk.corpus import stopwords
from gensim.models import ldamodel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

In [14]:
data=pd.read_csv('file:///home/mitra/IIITG/2ndsem/NPL/Assighment/project/abcnews-date-text.csv')
data

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1186013,20191231,vision of flames approaching corryong in victoria
1186014,20191231,wa police and government backflip on drug amne...
1186015,20191231,we have fears for their safety: victorian premier
1186016,20191231,when do the 20s start


In [17]:
data=data.sample(n=1000)
data

,publish_date,headline_text
103746,20040719,bracks tour encompasses mine visit
177767,20050725,airport security upgrade costs should be shared
683859,20120330,heart suffer colosimo injury blow
870464,20140429,brown coal injected engine
185565,20050831,wild winds whip canberra
...,...,...
138565,20050111,italian smokers forced outdoors
1005578,20160119,adelaide survived tsunami which never came 40 ...
114400,20040909,twelve iraqis killed in fresh us air strike on
1021162,20160415,accc cracks free range egg supplier


In [19]:
data_text = data[['headline_text']]
print(data_text)

                                             headline_text
103746                  bracks tour encompasses mine visit
177767     airport security upgrade costs should be shared
683859                   heart suffer colosimo injury blow
870464                          brown coal injected engine
185565                            wild winds whip canberra
...                                                    ...
138565                     italian smokers forced outdoors
1005578  adelaide survived tsunami which never came 40 ...
114400      twelve iraqis killed in fresh us air strike on
1021162                accc cracks free range egg supplier
819569                  minister stands strong on winching

[1000 rows x 1 columns]


In [41]:
data_text = data_text.astype('str')
print(data_text)
print(len(data_text))

                                             headline_text
103746   ["['bracks',", "'tour',", "'encompasses',", "'...
177767   ["['airport',", "'security',", "'upgrade',", "...
683859   ["['heart',", "'suffer',", "'colosimo',", "'in...
870464   ["['brown',", "'coal',", "'injected',", "'engi...
185565   ["['wild',", "'winds',", "'whip',", "'canberra...
...                                                    ...
138565   ["['italian',", "'smokers',", "'forced',", "'o...
1005578  ["['adelaide',", "'survived',", "'tsunami',", ...
114400   ["['twelve',", "'iraqis',", "'killed',", "'fre...
1021162  ["['accc',", "'cracks',", "'free',", "'range',...
819569   ["['minister',", "'stands',", "'strong',", "'w...

[1000 rows x 1 columns]
1000


In [42]:
for idx in range(len(data_text)):
    data_text.iloc[idx]['headline_text'] = [word for word in data_text.iloc[idx]['headline_text'].split(' ') if word not in stopwords.words()];
    if idx % 10 == 0:
        sys.stdout.write('\rc = ' + str(idx) + ' / ' + str(len(data_text)));
        pickle.dump(data_text, open('data_text.dat', 'wb'))
train_headlines = [value[0] for value in data_text.iloc[0:].values];

c = 990 / 1000

In [23]:
num_topics = 10

In [25]:
train_headlines_sentences = [' '.join(text) for text in train_headlines]

In [27]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000);
x_counts = vectorizer.fit_transform(train_headlines_sentences);

In [29]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [31]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [33]:
model = NMF(n_components=num_topics, init='nndsvd');
model.fit(xtfidf_norm)

NMF(init='nndsvd', n_components=10)

In [35]:
NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=10, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

NMF(init='nndsvd', n_components=10)

In [37]:
def get_nmf_topics(model, n_top_words):
    feat_names = vectorizer.get_feature_names()
    word_dict = {};
    for i in range(num_topics):
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    return pd.DataFrame(word_dict);

In [40]:
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,weather,nightmare,egyptsaikal,police,interview,indigenous,court,new,fire,warning
1,abc,bridge,schools,car,anthony,quarter,plane,highway,found,economy
2,wednesday,grafton,teacher,charges,griffin,ep4,crash,old,house,winter
3,tuesday,dud,children,hunt,kemp,australian,dead,push,toddler,sparks
4,news,bridgewater,underperforming,fatal,corey,service,australian,wa,two,national
5,business,closer,testing,bay,brett,public,wa,pm,guilty,spark
6,higgins,am1,grants,muswellbrook,ricky,staff,sentenced,govt,lab,burglaries
7,presenter,historic,monaro,wide,martin,bridge,lebanese,highlights,pool,spike
8,paul,urged,sporting,crash,ponting,myskina,three,australia,set,fires
9,reports,public,faces,rammed,parker,doubt,explosion,shipping,attacked,call
